In [ ]:
import pandas as pd
from pandas import DataFrame
import time
from yahoo_finance import Share
from datetime import timedelta
import numpy as np
from pytrends.request import TrendReq

google_username = ""
google_password = ""
path = ""

# connect to Google
pytrend = TrendReq(google_username, google_password, custom_useragent='My Pytrends Project')

def GTrendsMarket(location_index, keywords):
    
    for index in range(0, len(location_index)):  
        
        stock_index = Share(location_index.values()[index])
    
        stock_index_df = pd.DataFrame(stock_index.get_historical('2013-12-11', '2016-12-10'))
    
        stock_index_df = DataFrame(stock_index_df,columns=('Date', 'Close'))
    
        stock_index_df = stock_index_df.sort_values(by=['Date'], ascending=[True])
    
        stock_index_df = stock_index_df.reset_index()
    
        del stock_index_df['index']
    
        for row in keywords.iterrows():
    
            print("Downloading Keyword #: " + str(row[0]))
            print("Downloading Keyword name: " + row[1][0])
            trend_payload = {'geo': location_index.keys()[index], 'q': [row[0]], 'date': 'today 36-m'}   

            # trend
            df = pytrend.trend(trend_payload, return_type='dataframe')
    
            for i in range(1, 5):
                last_row = len(df) - 1
                df = df.drop(df.index[last_row]) 
    
    
            df.columns = ['{}_GT_score'.format(row[1][0])]
        
            df = df.reset_index()
        
            lt = []
    
            for i in range(0, len(df)):
                lt.append(str(df.iloc[i,0].date() + timedelta(days=1)))    

            del df['Date']
            df.insert(0, 'Date', lt)  
        
            stock_index_df = stock_index_df.merge(df,on='Date')
        
            time.sleep(180)
    
        stock_index_df.to_csv('{} {} Results.csv'.format(location_index.keys()[index], location_index.values()[index]), 
                              sep=',', encoding='utf-8')
        time.sleep(240)



# Specify the filename of a CSV with a list of keywords in the variable, keyordcsv. 
#The CSV should be one column, with header equal to Keywords (case sensitive).
keywordcsv = "pyt.csv"
keywords = pd.read_csv(keywordcsv, header=None)

location_index = {'US': '^DJI', 'IN':'^BSESN', 'GB':'^FTSE', 'AU':'^AXJO', 'BR':'^BVSP'}

GTrendsMarket(location_index, keywords)



res_csv_list = []
for index in range(0, len(location_index)):
    res_csv_list.append('{} {} Results.csv'.format(location_index.keys()[index], location_index.values()[index])) 
  
dff = pd.DataFrame(0, index = np.arange(len(res_csv_list)), columns = np.arange(len(keywords)))

p = 0

for i in res_csv_list:
    
    print i, "\n"
    data = pd.read_csv(i)
    q=0
    
    for row in keywords.iterrows():
        
        rtn=0
        print("Keyword name: " + row[1][0])
    
        df1 = data['{}_GT_score'.format(row[1][0])]
        df10 = data[['Close']]

        df111 = pd.DataFrame(0, index=np.arange(len(df1), 1), columns=np.arange(1))

        sell = 0 
        buy = 0
        invest = 100000
        
        print "Initial Investment : ",invest

        for i in range(0, 4):
            df111.ix[i,0] = 0
    
        for i in range(0, (len(df1)-4)):
            df111.ix[i+4,0] = df1.ix[i+3,0] - ( (df1.ix[i,0] + df1.ix[i+1,0] + df1.ix[i+2,0]) / 3 )
    
            if i != (len(df1)-5):
                if df111.ix[i+4,0] > 0: 
                    sell = df10.ix[i+4,0] * int(invest/df10.ix[i+4,0])
                    buy = df10.ix[i+5,0] * int(invest/df10.ix[i+4,0])
                    trade_PL = sell - buy
                    invest = invest + trade_PL
                elif df111.ix[i+4,0] < 0:
                    buy = df10.ix[i+4,0] * int(invest/df10.ix[i+4,0])
                    sell = df10.ix[i+5,0] * int(invest/df10.ix[i+4,0])
                    trade_PL = sell - buy
                    invest = invest + trade_PL
        
        print "Final Portfolio value : ",invest        
        
        rtn = ((invest-100000)/100000)*100
        
        print "Percentage Profit/Loss : ",((invest-100000)/100000)*100, "\n"

        dff.ix[p,q] = rtn
        q=q+1
    p=p+1

key_list = []

for row in keywords.iterrows():
    key_list.append(row[1][0])

dff.columns = key_list
dff.insert(0, 'Location', location_index.keys())
dff.to_csv('overall return.csv')


